# **나이브 베이즈 모델을 이용한 자소서 합/불 분류기**
Calssification

## **1 분류기 Classification**
1. **Binary Classification** (이진 분류기) : **True / False 조건을** 구분한다
1. **Multiclass Classification** (다변량 분류) : **다양한 클래스간의 조건을** 구분한다
1. **Multi-label Classification** (다중 클래스 레이블 분류) : 다중의 클래스간 **겹치는 조건에서** 구분을 한다

<img src="https://wikidocs.net/images/page/4291/logreg702.PNG" width=400>

## **2 텍스트 분류기 Classification**
1. **긍정/ 부정,  긍정/ 중립/ 부정** 분류기
1. **뉴스의 토픽** 분류기 (**class 간 중첩되어** 분류가 가능하다)
1. **Named Entity Recognition** (개체명 분류기) : ex) Naive Bayse, Support Vector Machine

## **3  Naive Bayse Classification 개념**
1. 확률 기반의 분류기
1. **Naive :** 예측을 위한 Token 들이 **Mutually Independent** (상호독립적)을 가정
1. **Bayse :** 관찰한 Token이 **클래스 전체 대비, 특정 클래스 속할 확률을 Bayse 기반** 으로 계산

> **Naive Bayse 메커니즘**

1. 스팸메일과, 정상메일로 구분된 데이터를 사용한다 [download](http://www.aueb.gr/users/ion/data/enron-spam/preprocessed/enron1.tar.gz)
1. 단어 **Token을** 대상으로 **스팸여부를** 학습한다.
1. Data 추가시 잘못 예측한 결과에 대해 **Laplace Smoothing** 으로 보완한 값을 **Bayse 로 공식을** 수정한다

## **4   Naive Bayse 구현하기**

<br>
### **01 enron 메일데이터 살펴보기**

1. **불합격자소서 (3,672개)** 와 **합격자소서 (1,500)개로** 약 1:2의 비율로 구분이 된다

In [1]:
# ham 폴더에 저장된 메일내용 확인 (정상으로 분류된 메일)
file_path = './enron1/pass/0001.caltex.pass.txt'
with open(file_path, 'rt', encoding='UTF8') as infile:
    ham_sample = infile.read()
print(ham_sample)

﻿"최고의 영업사원으로 성장하겠습니다"
GS칼텍스는 정유업을 강점으로 석유화학 및 윤활유에 이르기까지 계속된 성장을 거듭해 왔습니다. 향후 건설될 올레핀 생산시설은 GS칼텍스의 경쟁력을 더욱 높여줄 것입니다. 개인적으로 GS칼텍스가 이처럼 최고의 에너지 기업 자리를 유지하고 있는 원동력은 끊임없이 혁신을 추구하고, 최고가 되기 위해 노력했기 때문이라고 생각합니다.
이러한 GS칼텍스의 행보는 학부 시절 화공생명공학을 전공하고 관심이 있던 경영학을 대학원에서 배우며 도전했던 제 모습과 많이 닮아있다고 느꼈습니다. 또한 '매 순간 최선을 다해 최고에 도전하자'는 가치관을 가진 제게 큰 매력으로 다가왔습니다. GS칼텍스 영업직은 친화력과 추진력을 가진 저의 역량을 극대화할 수 있는 직무입니다. 또한, 제가 배웠던 것을 다방면으로 접목할 수 있는 최고의 기회입니다. 저는 정유업의 기초가 되는 화학공학 지식을 바탕으로 경영학에서 배운 비즈니스 사고를 접목해 최고의 영업사원으로 성장하겠습니다.
"차별화된 경쟁력-융합형 인재"
공학과 경영학을 공부하며 얻은 융합적 사고는 저만의 경쟁력이자 GS칼텍스 영업직에 가장 적합한 이유라고 생각합니다. 저는 학부 때 화공생명공학과를 전공하면서 석사로 경영학 마케팅을 공부했습니다. 융합에 관한 책을 읽은 이후 좀 더 넓은 안목을 가져야 할 필요성을 느꼈고 원하는 목표로 가는 과정에서 어떻게 다른 방식으로 접근하는지 알 수 있었습니다.
마케팅을 깊게 공부하면서 저는 데이터를 분석해 통계학적으로 소비자 행동을 관찰하였고, 더 나아가 숨겨진 메커니즘을 고민해보며 적절한 마케팅 전략을 수립하였습니다. 저는 이 과정에서 사람들을 설득하여 마음을 얻는 영업직에 자연스럽게 관심을 두게 되었습니다. 보스턴컨설팅그룹(BCG) 인턴을 하면서 저만의 논리를 가지고 클라이언트들을 설득한 경험은 최고의 영업 사원이 되고자 하는 제 꿈을 더욱 확고히 해주었습니다. 이러한 저의 경험과 노력은 GS칼텍스의 영업사원으로 업무를 하는데 큰 자산이 될 것입니다.
"편집장

In [2]:
# spam 폴더에 저장된 메일내용 확인 (스팸으로 분류된 메일)
file_path = './enron1/fail/0001.hyundai.fail.txt'
with open(file_path, 'rt', encoding='UTF8') as infile:
    spam_sample = infile.read()
print(spam_sample)

﻿"융합형 인재"
저는 학부 때 화공생명공학과를 전공하면서 석사로 경영학 마케팅을 공부했습니다. 융합에 관한 책을 읽은 이후 좀 더 넓은 안목을 가져야 할 필요성을 느꼈던 것이 계기였습니다. 이왕이면 본 전공과 전혀 다른 분야를 선택하고자 했습니다. 원하는 목표로 가는 과정에서 어떻게 다른 방식으로 접근하는지가 궁금했고 부전공을 통해 더욱 관심이 생긴 마케팅을 선택했습니다.
마케팅을 깊게 공부하면서 저는 데이터를 분석해 통계학적으로 소비자 행동을 관찰하였고 더 나아가 심리학을 기반으로 그 행동의 이유에 대해 고민해볼 수 있었습니다. 저는 이 과정에서 사람들을 설득하여 마음을 얻는 영업직에 자연스럽게 관심을 두게 되었습니다. 현대오일뱅크 영업직은 융합적 사고에 기반을 둔 제가 그동안 배웠던 것을 다방면으로 접목할 수 있는 최고의 기회라고 생각합니다. 저는 정유업의 기초가 되는 화학공학 지식을 바탕으로 경영학에서 배운 비즈니스 사고를 접목해 최고의 영업사원으로 성장하고 싶습니다.
"승부욕, 추진력, 도전"
저는 어렸을 때 강한 승부욕을 가진 청년이었습니다. 학창시절, 반 축구 대항전이나 계주를 나가면 이기기 위해 최선을 다하고 패배하면 큰 아쉬움에 친구들과 눈물을 보이곤 했습니다. 누구와 금방 친해질 수 있는 성격을 가졌던 저는 학창시절 줄곧 반장을 맡으면서 꽤 대담한 일에도 나아갈 수 있는 추진력을 길렀습니다. 대표적으로, 중학교 2학년 때 체육 담임선생님을 도와 반 친구들과 대전 마라톤 대회에 나간 경험을 소개하고 싶습니다. 저는 반장으로서 친구들과 계속해서 소통하고 힘든 시기를 함께 극복하며 모든 친구가 10km 마라톤을 완주하는 데에 큰 도움을 주었습니다. 고등학교 때에도 친구들과 함께 ‘시그마’라는 수학동아리를 만들어서 부족한 부분을 공유하고 다 같이 세미나를 여는 등 실력 배양을 위해 계속 도전을 하였습니다. 학창 시절의 이러한 경험들을 통해 저는 긍정적인 사고로 ‘매 순간 최선을 다해 최고에 도전하자’라는 제 가치관을 정립할 수 있었습니다.
"강인한 

### **02 enron 메일 데이터 분류하기**
1. 합격자소서와 불합격자소서를 레이블을 사용하여 분류한다
1. 1 : 합격자소서,  0 : 불합격자소서
1. 분류된 데이터를 전처리 과정을 진행한다

In [3]:
import glob, os
# 정상매일은 0, 스펨매일은 1
emails, labels = [], []
for no, file_path in enumerate(['./enron1/fail/','./enron1/pass/']):
    for filename in glob.glob(os.path.join(file_path, '*.txt')):
        try:
            with open(filename, 'rt', encoding='utf-8') as infile:  #line?? 
                emails.append(infile.read())
                labels.append(no)
        except UnicodeDecodeError as e:
            print(filename)
       

In [4]:
len(emails)

33

In [5]:
emails

['\ufeff"융합형 인재"\n저는 학부 때 화공생명공학과를 전공하면서 석사로 경영학 마케팅을 공부했습니다. 융합에 관한 책을 읽은 이후 좀 더 넓은 안목을 가져야 할 필요성을 느꼈던 것이 계기였습니다. 이왕이면 본 전공과 전혀 다른 분야를 선택하고자 했습니다. 원하는 목표로 가는 과정에서 어떻게 다른 방식으로 접근하는지가 궁금했고 부전공을 통해 더욱 관심이 생긴 마케팅을 선택했습니다.\n마케팅을 깊게 공부하면서 저는 데이터를 분석해 통계학적으로 소비자 행동을 관찰하였고 더 나아가 심리학을 기반으로 그 행동의 이유에 대해 고민해볼 수 있었습니다. 저는 이 과정에서 사람들을 설득하여 마음을 얻는 영업직에 자연스럽게 관심을 두게 되었습니다. 현대오일뱅크 영업직은 융합적 사고에 기반을 둔 제가 그동안 배웠던 것을 다방면으로 접목할 수 있는 최고의 기회라고 생각합니다. 저는 정유업의 기초가 되는 화학공학 지식을 바탕으로 경영학에서 배운 비즈니스 사고를 접목해 최고의 영업사원으로 성장하고 싶습니다.\n"승부욕, 추진력, 도전"\n저는 어렸을 때 강한 승부욕을 가진 청년이었습니다. 학창시절, 반 축구 대항전이나 계주를 나가면 이기기 위해 최선을 다하고 패배하면 큰 아쉬움에 친구들과 눈물을 보이곤 했습니다. 누구와 금방 친해질 수 있는 성격을 가졌던 저는 학창시절 줄곧 반장을 맡으면서 꽤 대담한 일에도 나아갈 수 있는 추진력을 길렀습니다. 대표적으로, 중학교 2학년 때 체육 담임선생님을 도와 반 친구들과 대전 마라톤 대회에 나간 경험을 소개하고 싶습니다. 저는 반장으로서 친구들과 계속해서 소통하고 힘든 시기를 함께 극복하며 모든 친구가 10km 마라톤을 완주하는 데에 큰 도움을 주었습니다. 고등학교 때에도 친구들과 함께 ‘시그마’라는 수학동아리를 만들어서 부족한 부분을 공유하고 다 같이 세미나를 여는 등 실력 배양을 위해 계속 도전을 하였습니다. 학창 시절의 이러한 경험들을 통해 저는 긍정적인 사고로 ‘매 순간 최선을 다해 최고에 도전하자’라는 제 가치관을 정립할 수 있

In [87]:
from konlpy.tag import Okt
okt = Okt()

text = "찍어야지"

print(okt.morphs(text))
print(okt.morphs(text, stem=True))

['찍어야지']
['찍다']


In [88]:
z3=[]
for word in z1:
    a=okt.morphs(word, stem=True)
    for s in a:
        z3.append(s)


In [96]:
z3=[]
for word in z1:
    a=okt.pos(word, stem=True)
    for s in a:
        z3.append(s)


In [97]:
z3

[('"', 'Punctuation'),
 ('융합', 'Noun'),
 ('형', 'Suffix'),
 ('인재', 'Noun'),
 ('"', 'Punctuation'),
 ('저', 'Noun'),
 ('는', 'Josa'),
 ('학부', 'Noun'),
 ('때', 'Noun'),
 ('화공', 'Noun'),
 ('생명', 'Noun'),
 ('공학', 'Noun'),
 ('과를', 'Josa'),
 ('전공', 'Noun'),
 ('하다', 'Verb'),
 ('석사', 'Noun'),
 ('로', 'Josa'),
 ('경영학', 'Noun'),
 ('마케팅', 'Noun'),
 ('을', 'Josa'),
 ('공부', 'Noun'),
 ('하다', 'Verb'),
 ('융합', 'Noun'),
 ('에', 'Josa'),
 ('관', 'Noun'),
 ('한', 'Josa'),
 ('책', 'Noun'),
 ('을', 'Josa'),
 ('읽다', 'Verb'),
 ('이후', 'Noun'),
 ('좀', 'Noun'),
 ('더', 'Noun'),
 ('넓다', 'Adjective'),
 ('안목', 'Noun'),
 ('을', 'Josa'),
 ('가다', 'Verb'),
 ('하다', 'Verb'),
 ('필요성', 'Noun'),
 ('을', 'Josa'),
 ('느끼다', 'Verb'),
 ('것', 'Noun'),
 ('이', 'Josa'),
 ('계기', 'Noun'),
 ('이다', 'Verb'),
 ('이왕이면', 'Noun'),
 ('보다', 'Verb'),
 ('전공', 'Noun'),
 ('과', 'Josa'),
 ('전혀', 'Noun'),
 ('다른', 'Noun'),
 ('분야', 'Noun'),
 ('를', 'Josa'),
 ('선택', 'Noun'),
 ('하다', 'Verb'),
 ('하다', 'Verb'),
 ('원하다', 'Adjective'),
 ('목표', 'Noun'),
 ('로', 'Josa'),
 ('

In [84]:
# 어간 추출
from konlpy.tag import Okt
okt = Okt()

print(okt.morphs(z1))


TypeError: No matching overloads found for kr.lucypark.okt.OktInterface.tokenize(list,java.lang.Boolean,java.lang.Boolean), options are:
	public java.util.List kr.lucypark.okt.OktInterface.tokenize(java.lang.String,java.lang.Boolean,java.lang.Boolean)

	at JPMethod::findOverload(native\common\jp_method.cpp:242)
	at JPMethod::findOverload(native\common\jp_method.cpp:245)
	at JPMethod::invoke(native\common\jp_method.cpp:253)
	at PyJPMethod::__call__(native\python\pyjp_method.cpp:142)


In [ ]:
print(okt.morphs(z1, stem=True))

In [35]:
import re

In [42]:
#불필요한 단어 제거! 
z1 = []
import re
for ch in z:
    z1.append(re.sub('[-?.\ufeff]','',ch))
    

In [43]:
z1

['" 융합형 인재" 저는 학부 때 화공생명 공학과를 전공하면서 석사로 경영학 마케팅을 공부했습니다',
 '융합에 관한 책을 읽은 이후 좀 더 넓은 안목을 가져야 할 필요성을 느꼈던 것이 계기였습니다',
 '이왕이면 본 전공과 전혀 다른 분야를 선택하고자 했습니다',
 '원하는 목표로 가는 과정에서 어떻게 다른 방식으로 접근하는 지가 궁금했고 부 전공을 통해 더욱 관심이 생긴 마케팅을 선택했습니다',
 '마케팅을 깊게 공부하면서 저는 데이터를 분석해 통계학적으로 소비자 행동을 관찰하였고 더 나 아가 심리학을 기반으로 그 행동의 이유에 대해 고민해 볼 수 있었습니다',
 '저는 이 과정에서 사람들을 설득하여 마음을 얻는 영업직에 자연스럽게 관심을 두게 되었습니다',
 '현대 오일 뱅크 영업직은 융합적 사고에 기반을 둔 제가 그동안 배웠던 것을 다방면으로 접목할 수 있는 최고의 기회라고 생각합니다',
 '저는 정 유업의 기초가 되는 화학공학 지식을 바탕으로 경영학에서 배운 비즈니스 사고를 접목해 최고의 영업사원으로 성장하고 싶습니다',
 '" 승부욕, 추진력, 도전" 저는 어렸을 때 강한 승부욕을 가진 청년이었습니다',
 '학창시절, 반 축구 대항전이나 계주를 나가면 이기기 위해 최선을 다하고 패배하면 큰 아쉬움에 친구들과 눈물을 보이곤 했습니다',
 '누구와 금방 친해 질 수 있는 성격을 가졌던',
 '저는 학창시절 줄곧 반장을 맡으면서 꽤 대담한 일에도 나아갈 수 있는 추진력을 길렀습니다',
 '대표적으로, 중학교 2 학년 때 체육 담임 선생님을 도와 반 친구들과 대전 마라톤 대회에 나간 경험을 소개하고 싶습니다',
 '저는 반장으로서 친구들과 계속해서 소통하고 힘든 시기를 함께 극복하며 모든 친구가 10km 마라톤을 완주하는 데에 큰 도움을 주었습니다',
 '고등학교 때에도 친구들과 함께 ‘ 시그마’ 라는 수학 동아리를 만들어서 부족한 부분을 공유하고 다 같이 세미나를 여는 등 실력 배양을 위해 계속 도전을 하였습니다',
 '학창 시절의 이러한 경험들을 통해

In [64]:
#불융어 
stop = []
with open('./enron1/stopword.txt', 'rt', encoding='UTF8') as f:    #./?
    while True:
        line= f.readline()
        stop.append(line)
        if not line: break
    
    
                
print(stop)

['아\n', '휴\n', '아이구\n', '아이쿠\n', '아이고\n', '어\n', '나\n', '우리\n', '저희\n', '따라\n', '의해\n', '을\n', '를\n', '에\n', '의\n', '가\n', '으로\n', '로\n', '에게\n', '뿐이다\n', '의거하여\n', '근거하여\n', '입각하여\n', '기준으로\n', '예하면\n', '예를 들면\n', '예를 들자면\n', '저\n', '소인\n', '소생\n', '저희\n', '지말고\n', '하지마\n', '하지마라\n', '다른\n', '물론\n', '또한\n', '그리고\n', '비길수 없다\n', '해서는 안된다\n', '뿐만 아니라\n', '만이 아니다\n', '만은 아니다\n', '막론하고\n', '관계없이\n', '그치지 않다\n', '그러나\n', '그런데\n', '하지만\n', '든간에\n', '논하지 않다\n', '따지지 않다\n', '설사\n', '비록\n', '더라도\n', '아니면\n', '만 못하다\n', '하는 편이 낫다\n', '불문하고\n', '향하여\n', '향해서\n', '향하다\n', '쪽으로\n', '틈타\n', '이용하여\n', '타다\n', '오르다\n', '제외하고\n', '이 외에\n', '이 밖에\n', '하여야\n', '비로소\n', '한다면 몰라도\n', '외에도\n', '이곳\n', '여기\n', '부터\n', '기점으로\n', '따라서\n', '할 생각이다\n', '하려고하다\n', '이리하여\n', '그리하여\n', '그렇게 함으로써\n', '하지만\n', '일때\n', '할때\n', '앞에서\n', '중에서\n', '보는데서\n', '으로써\n', '로써\n', '까지\n', '해야한다\n', '일것이다\n', '반드시\n', '할줄알다\n', '할수있다\n', '할수있어\n', '임에 틀림없다\n', '한다면\n', '등\n', '등등\n', '제\n', '겨우\n', '단지\n', '다만\n', '할뿐\n', '딩동\n

In [65]:
stop1 = []
for ch in stop:
    stop1.append(re.sub('[\n]','',ch))
    

In [67]:
len(stop1)

676

In [6]:
#패키지 이용하기
#!pip 
from konlpy.tag import Twitter
from konlpy.tag import Kkma
kkma= Kkma()

C:\Users\bgmm0\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [7]:
kkma.sentences('한국어 분석을 시작합니다 안녕하세요~~')

['한국어 분석을 시작합니다', '안녕 하세요~~']

In [8]:
kkma.pos('한국어 분석을 시작합니다 안녕하세요~~')

[('한국어', 'NNG'),
 ('분석', 'NNG'),
 ('을', 'JKO'),
 ('시작하', 'VV'),
 ('ㅂ니다', 'EFN'),
 ('안녕', 'NNG'),
 ('하세', 'NNG'),
 ('요', 'JX'),
 ('~~', 'SW')]

In [9]:
#문장단위로 쪼개보기!
z=[]
for email in emails:
    sp = kkma.sentences(email)
    for sen in sp:
        z.append(sen)


In [13]:
z.index('\ufeff" 최고의 영업사원으로 성장하겠습니다')

817

In [14]:
z[817]

'\ufeff" 최고의 영업사원으로 성장하겠습니다'

In [17]:
type(labels[0])

int

In [44]:
labels=[]
for i in range(817):
    labels.append(0)
for i in range(415):
    labels.append(1)
    

In [45]:
len(labels)

1232

### **03 enron 메일 데이터 임베딩**
1. Chapter 2 에서 진행한 내용을 바탕으로 전처리 작업을 진행한다
1. **숫자와 구두점** 제거, **StopWords** 제거, **표제어 원형** 복원
1. 정제된 데이터로 **희소벡터 (Sparse Vector)** 로 임베딩 ex) (**row index, feacture/term index**)

[Install_nltk](https://www.nltk.org/install.html)

[download_nltk_data](https://stackoverflow.com/questions/35861482/nltk-lookup-error)

```
import nltk
nltk.download('all')
```

In [25]:
#download_nltk_data
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\bgmm0\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\bgmm0\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\bgmm0\AppData\Roaming\nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\bgmm0\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\Users\bgmm0\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     C:\Users\b

[nltk_data]    |   Package qc is already up-to-date!
[nltk_data]    | Downloading package reuters to
[nltk_data]    |     C:\Users\bgmm0\AppData\Roaming\nltk_data...
[nltk_data]    |   Package reuters is already up-to-date!
[nltk_data]    | Downloading package rte to
[nltk_data]    |     C:\Users\bgmm0\AppData\Roaming\nltk_data...
[nltk_data]    |   Package rte is already up-to-date!
[nltk_data]    | Downloading package semcor to
[nltk_data]    |     C:\Users\bgmm0\AppData\Roaming\nltk_data...
[nltk_data]    |   Package semcor is already up-to-date!
[nltk_data]    | Downloading package senseval to
[nltk_data]    |     C:\Users\bgmm0\AppData\Roaming\nltk_data...
[nltk_data]    |   Package senseval is already up-to-date!
[nltk_data]    | Downloading package sentiwordnet to
[nltk_data]    |     C:\Users\bgmm0\AppData\Roaming\nltk_data...
[nltk_data]    |   Package sentiwordnet is already up-to-date!
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     C:\Users\bg

True

In [98]:
stop1

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간',
 '약간',
 '다소',
 '좀',
 '조

In [76]:
%%time
# 출현빈도가 높은 상위 500개의 Token을 대상으로 임베딩 한다
# 희소벡터(Sparse Vector)로 변환 : (row index, feacture/term index)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words=stop1, max_features=500)#맞나??.. 
term_docs      = cv.fit_transform(z)
print("모델의 Type: {}\n임베딩의 크기: {}\n0번문장 내용보기: \n{}".format(
    type(term_docs),
    term_docs.shape, # 5,172개 문장을 500개 단어로 생성
    term_docs [2]))  # 0번 문장의 단어 Vector 목록을 출력   #0이 안나옴!!  #개수도 5172개 문장이 아닌디? 

모델의 Type: <class 'scipy.sparse.csr.csr_matrix'>
임베딩의 크기: (1232, 500)
0번문장 내용보기: 
  (0, 482)	1
Wall time: 50 ms


C:\Users\bgmm0\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['가서', '같은', '것과', '결과에', '결론을', '관계가', '관련이', '그런', '그럼에도', '그렇게', '그에', '그치지', '김에', '까닭에', '낫다', '년도', '논하지', '누가', '다시', '달려', '대로', '대해', '되는', '되다', '되어', '들면', '들자면', '듯하다', '따르는', '따름이다', '따지지', '때가', '만은', '만이', '만큼', '많은', '말하면', '말할것도', '몰라도', '몰랏다', '못하다', '미치다', '바꾸어서', '바꿔', '밖에', '방면으로', '보면', '보아', '부류의', '비길수', '비추어', '뿐만', '사람들', '상대적으로', '생각이다', '서술한바와같이', '쓰여', '아니다', '아니라', '안다', '안된다', '않고', '않기', '않는다면', '않다', '않다면', '않도록', '않으면', '알겠는가', '어쩔수', '어찌됏든', '없고', '없다', '예를', '외에', '요만한', '우에', '위에서', '이렇게', '이로', '이르다', '이와', '이유는', '인하여', '임에', '점에서', '정도에', '정도의', '종합한것과같이', '주저하지', '줄은', '지경이다', '틀림없다', '편이', '하고', '하기', '하기만', '하는', '하는것만', '하는것이', '하다', '하면', '하지', '한하다', '할수록', '함으로써', '해도', '해서는', '형식으로', '힘이'] not in stop_words.
  'stop_words.' % sorted(inconsist

In [105]:
%%time
# 출현빈도가 높은 상위 500개의 Token을 대상으로 임베딩 한다
# 희소벡터(Sparse Vector)로 변환 : (row index, feacture/term index)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words=stop1, max_features=500)
term_docs      = cv.fit_transform(z1)
print("모델의 Type: {}\n임베딩의 크기: {}\n3번문장 내용보기: \n{}".format(
    type(term_docs),
    term_docs.shape, # 1,232개 문장을 500개 단어로 생성
    term_docs [3]))  # 2번 문장의 단어 Vector 목록을 출력  

모델의 Type: <class 'scipy.sparse.csr.csr_matrix'>
임베딩의 크기: (1232, 500)
3번문장 내용보기: 
  (0, 236)	1
  (0, 232)	1
  (0, 65)	1
  (0, 117)	1
  (0, 441)	1
  (0, 392)	1
  (0, 367)	1
  (0, 189)	1
  (0, 59)	1
  (0, 15)	1
  (0, 174)	1
  (0, 302)	1
  (0, 152)	1
Wall time: 42.9 ms


In [94]:
len(cv.stop_words_)

4092

In [95]:
cv.vocabulary_

{'융합형': 312,
 '저는': 353,
 '학부': 471,
 '전공하면서': 363,
 '경영학': 37,
 '마케팅을': 152,
 '공부했습니다': 56,
 '융합에': 310,
 '책을': 407,
 '이후': 325,
 '넓은': 96,
 '가져야': 17,
 '필요성을': 462,
 '것이': 31,
 '했습니다': 482,
 '원하는': 302,
 '목표로': 174,
 '가는': 15,
 '과정에서': 59,
 '방식으로': 189,
 '접근하는': 367,
 '지가': 392,
 '통해': 441,
 '더욱': 117,
 '관심이': 65,
 '생긴': 232,
 '선택했습니다': 236,
 '데이터를': 120,
 '분석해': 209,
 '소비자': 242,
 '행동을': 484,
 '아가': 271,
 '대해': 115,
 '고민해': 54,
 '있었습니다': 345,
 '자연스럽게': 348,
 '관심을': 64,
 '두게': 135,
 '되었습니다': 134,
 '제가': 372,
 '그동안': 71,
 '것을': 30,
 '접목할': 369,
 '있는': 334,
 '최고의': 412,
 '기회라고': 87,
 '생각합니다': 229,
 '되는': 132,
 '바탕으로': 179,
 '비즈니스': 215,
 '사고를': 218,
 '영업사원으로': 293,
 '싶습니다': 269,
 '가진': 19,
 '학창시절': 474,
 '위해': 305,
 '최선을': 414,
 '친구들과': 421,
 '친해': 422,
 '성격을': 237,
 '대담한': 111,
 '추진력을': 417,
 '중학교': 390,
 '대회에': 116,
 '경험을': 41,
 '소개하고': 241,
 '계속해서': 46,
 '모든': 172,
 '친구가': 420,
 '데에': 118,
 '도움을': 123,
 '주었습니다': 385,
 '고등학교': 53,
 '라는': 146,
 '수학': 252,
 '만들어서': 156,
 '부분을': 201,
 '

In [70]:
print(cv.get_stop_words)

<bound method VectorizerMixin.get_stop_words of CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=500, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)>


In [81]:
term_docs

<1232x500 sparse matrix of type '<class 'numpy.int64'>'
	with 9085 stored elements in Compressed Sparse Row format>

In [102]:
# cv 모델로 인덱스별 단어 Token 내용보기
# feature_mapping = cv.vocabulary_       # dict 로 내용출력 (key:value)

feature_names   = cv.get_feature_names() # List 로 내용출력 (인덱스별 value)
for indx in [53, 162, 481, 357, 125]:
    print(indx, ":", feature_names[indx])

53 : 고등학교
162 : 매거진
481 : 해당
357 : 적극적으로
125 : 도전하자


In [99]:
for indx in range(500):
    print(indx, ":", feature_names[indx])

0 : 2018
1 : 2019
2 : 2년
3 : 4차
4 : 52
5 : bcg
6 : forest
7 : gs
8 : lg
9 : nice
10 : python
11 : python을
12 : random
13 : regression
14 : sas
15 : sns
16 : 가는
17 : 가장
18 : 가져야
19 : 가지고
20 : 가진
21 : 가치관을
22 : 가치를
23 : 간의
24 : 강점인
25 : 강조한
26 : 같은
27 : 개인적으로
28 : 거래
29 : 것에
30 : 것으로
31 : 것을
32 : 것이
33 : 것이라고
34 : 것이었습니다
35 : 것입니다
36 : 결과
37 : 결과를
38 : 경영
39 : 경영학
40 : 경영학을
41 : 경험
42 : 경험은
43 : 경험을
44 : 경험이
45 : 경험하고
46 : 계량
47 : 계속해서
48 : 고객
49 : 고객들의
50 : 고객에게
51 : 고객을
52 : 고객의
53 : 고객이
54 : 고등학교
55 : 고민해
56 : 공공
57 : 공부했습니다
58 : 공유하고
59 : 과의
60 : 과정에서
61 : 과정을
62 : 과제
63 : 관련
64 : 관련된
65 : 관심을
66 : 관심이
67 : 관점에서
68 : 구매
69 : 구현해
70 : 국내
71 : 그동안
72 : 그룹
73 : 긍정적인
74 : 기반
75 : 기법을
76 : 기부에
77 : 기부할
78 : 기술을
79 : 기업
80 : 기업의
81 : 기업이
82 : 기여하고
83 : 기존
84 : 기존에
85 : 기한에
86 : 기한이
87 : 기회라고
88 : 기획
89 : 기획을
90 : 기획하는
91 : 깊은
92 : 꿈을
93 : 끊임없이
94 : 내는
95 : 내적
96 : 너무
97 : 넓은
98 : 노력했습니다
99 : 높은
100 : 느꼈고
101 : 느꼈습니다
102 : 느낀
103 : 느낄
104 : 능력을
105 : 다가왔습니다
106 : 다루는
107 : 다르게
108 : 다양한
109

In [49]:
In [1]: import numpy as np

   ...: import pandas as pd

   ...: import os



In [2]: base_dir = "C:/Users/bgmm0/Desktop/대학원/머신러닝과 응용/프로젝트"

   ...: file_nm = "df.xlsx"

   ...: xlxs_dir = os.path.join(base_dir, file_nm) 



In [53]:
In [3]: df = pd.DataFrame({

   ...: 'value_1': labels,

   ...: 'value_2': z},

   ...: ) #index = [1, 2, 3, 4, 5]b
    



In [52]:
[1:20]

SyntaxError: invalid syntax (<ipython-input-52-bc5f7d8a71cb>, line 1)

In [54]:
df

value_1                                            value_2
0           0  ﻿" 융합형 인재" 저는 학부 때 화공생명 공학과를 전공하면서 석사로 경영학 마케팅...
1           0  융합에 관한 책을 읽은 이후 좀 더 넓은 안목을 가져야 할 필요성을 느꼈던 것이 계...
2           0                   이왕이면 본 전공과 전혀 다른 분야를 선택하고자 했습니다.
3           0  원하는 목표로 가는 과정에서 어떻게 다른 방식으로 접근하는 지가 궁금했고 부 전공을...
4           0  마케팅을 깊게 공부하면서 저는 데이터를 분석해 통계학적으로 소비자 행동을 관찰하였고...
5           0  저는 이 과정에서 사람들을 설득하여 마음을 얻는 영업직에 자연스럽게 관심을 두게 되...
6           0  현대 오일 뱅크 영업직은 융합적 사고에 기반을 둔 제가 그동안 배웠던 것을 다방면으...
7           0  저는 정 유업의 기초가 되는 화학공학 지식을 바탕으로 경영학에서 배운 비즈니스 사고...
8           0       " 승부욕, 추진력, 도전" 저는 어렸을 때 강한 승부욕을 가진 청년이었습니다.
9           0  학창시절, 반 축구 대항전이나 계주를 나가면 이기기 위해 최선을 다하고 패배하면 큰...
10          0                           누구와 금방 친해 질 수 있는 성격을 가졌던
11          0  저는 학창시절 줄곧 반장을 맡으면서 꽤 대담한 일에도 나아갈 수 있는 추진력을 길렀...
12          0  대표적으로, 중학교 2 학년 때 체육 담임 선생님을 도와 반 친구들과 대전 마라톤 ...
13          0  저는 반장으로서 친구들과 계속해서 소통하고 힘든 시기를 함께 극복하며 모든 친구가 ...
14          0  고등학교 때에도 친구들과 함께 ‘ 시그마’ 라는 수학 동아리를 만들어서 부족한 부분...
15          0  학창 시절의 이러한 경험들을 통해 저는 긍정적인 사고로 ‘ 매 순간 최선을 다해 최...
16          0  " 강인한 추진력" 저는 학교 스포츠 매거진 시스 붐 바의 리더 인 편집장을 맡아 ...
17          0  제가 편집장이 되었을 당시, 시스 붐 바 재정에 문제가 생기면서 잡지를 낼 수 없는...
18          0  저희는 자체 마케팅을 통해 후원을 받아야 했고 학교 운동부를 후원하던 언더 아 머 ...
19          0  스포츠라는 공통 분모도 있었고 대학 스포츠 쪽으로 영역을 넓힐 계획을 하던 언더 아...
20          0                                            생각했습니다.
21          0                   운이 좋게 언더 아 머로부터 PT 발표를 제안 받았습니다.
22          0  발표에서 시스 붐 바가 언더 아 머와 함께 할 수 있는 여러 프로젝트를 제안했고 긍...
23          0  일정의 후원금을 받으면서 언더 아 머가 계획하던 대학 스포츠 활성화 프로젝트에 도움...
24          0  단체의 리더로서 위기를 극복하고 관계자들과 성공적인 관계를 형성한 이 일은 저에게 ...
25          0  ﻿[ 최고 수준의 데이터 분석가] KT는 국내 ICT 산업을 주도하며 혁신기술을 기...
26          0  저는 그 원동력으로 KT 가 근성을 가지고 최고가 되기 위해 끊임없이 노력했기 때문...
27          0  이 점은 ' 매 순간 최선을 다해 최고에 도전하자' 는 가치관을 가진 제게 큰 매력...
28          0  미디어 통계 분석 직무는 KT의 축적된 데이터를 분석하여 차별화된 서비스를 제공하고...
29          0  해 당 직무는 계량 마케팅 연구실에서 통계 분석 툴을 사용해 실무자들에게 여러 인사...
...       ...                                                ...
1202        1  둘째, 정량 데이터 분석 뿐만 아니라 텍스트와 같은 비정형 데이터 처리에 대한 능력...
1203        1           정형 데이터 뿐만 아니라 비정형 데이터에 대한 중요성이 커지고 있습니다.
1204        1  CS에서의 고객 음성 데이터 등을 활용하여 새로운 서비스를 개발하는 데 기여하고 싶...
1205        1  마지막으로 빅 데이터 학회에서 SQL를 이용하여 빅 데이터를 다루는 엔지니어링 기술...
1206        1  회사 내에서 구성원들 과의 스터디를 통하여 대용량 데이터 처리, AI, 머신 러닝,...
1207        1  데이터 분석을 통해 insight를 찾아 새로운 비즈니스 모델에 기여하는 것이 설레...
1208        1  빅 데이터 분석 전문가가 되기 위해서는 해당 산업에 대한 배경지식이 폭넓어야 했기에...
1209        1  통계학과의 전공 수업에서는 주로 통계적인 방법론을 수학하여 여러 산업에 대한 데이터...
1210        1  다양한 산업과 형태의 로그 데이터를 다루고 데이터에서 의미를 도출하는 역량을 키우고...
1211        1  빅 데이터를 다루는 마케팅 연구실에서 R, Python, SAS를 이용하여 커머스와...
1212        1  빅 데이터 수업 및 빅 데이터 학회에서 정량 데이터 뿐만 아니라 비 정량 데이터 분...
1213        1  기존에는 현업에 대해 설문을 조사하는 것이 일반적인 틀이었고 마케팅에서 정책에 대한...
1214        1  이에 현업에 대한 빅 데이터를 분석하고, 설문으로 보완하는 방법을 통해 기존과 다른...
1215        1  버스 산업 실무 데이터에 대하여 52 시간 제도가 미치는 영향을 새로운 방법으로 분...
1216        1  대체적으로 52 시간 제도의 긍정적인 부분만을 언급했지만, 52 시간 제도로 인해 ...
1217        1  중국에서 자국민들이 위 키 피디아에서 지식을 공유하는 것을 차단한 적이 있었고, 이...
1218        1  주변에서는 기업마다 52 시간 제도의 시행이 달라 정확한 효과를 찾아내기 어려울 거...
1219        1  실제 현업의 데이터는 정제가 깔끔하지 않고 노이즈가 많아 52 시간 제도 만의 효과...
1220        1  52 시간 제도의 효과를 보기 위해 실제 버스 회사의 협조를 얻어서 실시간으로 수집...
1221        1  그리고 버스 기사님들과 본사 직원과 버스를 이용하는 소비자의 만족도를 비교하기 위해...
1222        1  버스 운행에 대한 현업을 제대로 파악하고자 지방에 있는 버스 회사로 찾아가 본사 직...
1223        1  52 시간 제도에 따른 노선 변경으로 인해 버스 기사님과 소비자들의 삶의 질에 영향...
1224        1  [ 관련 경험 1] ( 필수) (100 자 2 단락 이내) - 기상청 주관 2018...
1225        1  - 데이터 분석과 마케팅 역량을 활용하여 해당 주제가 미치는 영향력에 대해 고찰했습니다.
1226        1               - 빅 데이터를 활용하여 기획해 보는 경험을 얻을 수 있었습니다.
1227        1  [ 상 세 내용 2] ( 필수) (300 자 7 단락 이내) 빅 데이터 분석 프로그...
1228        1          - 데이터를 토픽 모델링을 적용하여 학습하고 네트워크 분석을 진행했습니다.
1229        1  - 토픽모델로 나온 결과를 해석하여 토픽의 개수를 선택하고 주제를 선택하는 부분이 ...
1230        1  [ 관련 경험 2] ( 필수) (100 자 2 단락 이내) 집 값 예측 Kaggle...
1231        1  - 모델의 parameter를 최적으로 맞춰 가

In [55]:
#-- write an object to an Excel sheet using pd.DataFrame.to_excel()

df.to_excel(xlxs_dir, # directory and file name to write

            sheet_name = 'Sheet1', 

            na_rep = 'NaN', 

            float_format = "%.2f", 

            header = True, 

            #columns = ["group", "value_1", "value_2"], # if header is False

            index = True, 

            index_label = "id", 

            startrow = 1, 

            startcol = 1, 

            #engine = 'xlsxwriter', 

            freeze_panes = (2, 0)

            ) 


In [56]:
df.to_excel() 

TypeError: to_excel() missing 1 required positional argument: 'excel_writer'

### **04-1 Naive Bayse 학습을 위한 준비작업**
모델의 학습을 위한 준비작업으로 데이터를 그룹화 한다

In [134]:
# 레이블을 기준으로 데이터를 그룹화 한다
# defaultdict : 스팸여부 0,1 Tag 로 Token Index List 생성
def get_label_index(labels):
    from collections import defaultdict
    label_index = defaultdict(list)
    for index, label in enumerate(labels):
        label_index[label].append(index)
    return label_index

# 0 ~ 3600 : 정상메일[0], 3600 ~ 나머지 : 스팸메일[1]
label_index = get_label_index(labels)
print(label_index.keys())
label_index[1][:10]

dict_keys([0, 1])


[3672, 3673, 3674, 3675, 3676, 3677, 3678, 3679, 3680, 3681]